In [265]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb

from sklearn.model_selection import GridSearchCV
from sklearn.metrics import roc_auc_score


In [266]:
df = pd.read_csv(f'/content/in_actions.csv', index_col=0)
df_rfm = df.copy() # заготовим датасет для последущей RFM-сегментации

In [267]:
df.head().T

id,1,2,3,4,5
vydacha_dt,2019-01-30 00:00:00.000,2021-11-12 00:00:00.000,2021-06-17 00:00:00.000,2021-11-08 00:00:00.000,2021-11-28 00:00:00.000
activation_dt,2021-11-22 00:00:00.000,2021-11-12 00:00:00.000,2021-11-22 00:00:00.000,2021-11-08 00:00:00.000,2021-11-28 00:00:00.000
pro0,1,0,1,1,1
pro1,1,0,1,1,1
pro2,1,1,1,1,1
chastotta,secondhalf,firsthalf,secondhalf,secondhalf,secondhalf
tran_type_month0,CRED,CRED+DEB,CRED,CRED+DEB,CRED
tran_type_month1,CRED+DEB,CRED+DEB,CRED,CRED+DEB,CRED
tran_type_month2,CRED,CRED,CRED,CRED+DEB,CRED
FREE_LIMIT0,3928.86,14520.0,14869.29,90000.0,44950.0


In [268]:
# Проведем чистку данных и переведем в необходимый формат
df.withdraw0 = df.withdraw0.str.replace(',', '.').astype('float')
df.withdraw1 = df.withdraw1.str.replace(',', '.').astype('float')
df.withdraw2 = df.withdraw2.str.replace(',', '.').astype('float')

In [269]:
# переведем даты в формат UNIX
df['vydacha_dt'] = pd.to_datetime(df['vydacha_dt']).map(pd.Timestamp.timestamp)
df['activation_dt'] = pd.to_datetime(df['activation_dt']).map(pd.Timestamp.timestamp)

In [270]:
df.dtypes

vydacha_dt                    float64
activation_dt                 float64
pro0                            int64
pro1                            int64
pro2                            int64
chastotta                      object
tran_type_month0               object
tran_type_month1               object
tran_type_month2               object
FREE_LIMIT0                   float64
FREE_LIMIT1                   float64
FREE_LIMIT2                   float64
limit                         float64
login_0                       float64
login_1                       float64
login_2                       float64
withdraw0                     float64
withdraw1                     float64
withdraw2                     float64
cred_tran_nopartnershare0     float64
cred_tran_nopartnershare1     float64
cred_tran_nopartnershare2     float64
cl_age                          int64
GENDER_ID                     float64
tran_prod                     float64
tran_remont                   float64
tran_inet   

In [271]:
# коэффициент оттока достаточно высокий
df.churn.value_counts(normalize=True)

0    0.538959
1    0.461041
Name: churn, dtype: float64

In [272]:
# оценим и отсортируем коэффициенты корреляции. Отрицательные значения в 'values' означают обратную корреляцию
df_cor = df.drop(['churn'], axis=1).corrwith(df.churn).to_frame('values')
df_cor = df_cor.assign(sorted = df_cor['values'].abs())
df_cor.sort_values(['sorted'], ascending=False)

,values,sorted
count_tran2,-0.311292,0.311292
cred_tran_nopartnershare0,0.275961,0.275961
counttran1,-0.219043,0.219043
tran_prod,-0.191807,0.191807
tran_amt2,-0.146080,0.146080
CRED_TRAN_TOTAL,0.133181,0.133181
CREDIT_TRAN_AMT_last_month,-0.130355,0.130355
login_1,-0.127832,0.127832
login_0,-0.123023,0.123023
tran_inet,-0.121541,0.121541


In [273]:
# проведем подготовку датасета для моделирования. Унифицируем данные. Переведем категориальные переменные в числовые
df.columns = df.columns.str.lower().str.replace(' ', '_')

categorical_columns = list(df.dtypes[df.dtypes == 'object'].index)

for c in categorical_columns:
    df[c] = df[c].str.lower().str.replace(' ', '_')

In [274]:
df.isnull().sum()

vydacha_dt                        0
activation_dt                     0
pro0                              0
pro1                              0
pro2                              0
chastotta                         0
tran_type_month0                  0
tran_type_month1                  0
tran_type_month2                  0
free_limit0                       2
free_limit1                       2
free_limit2                       2
limit                             0
login_0                         116
login_1                         585
login_2                        1062
withdraw0                         0
withdraw1                         0
withdraw2                         0
cred_tran_nopartnershare0      6923
cred_tran_nopartnershare1      4853
cred_tran_nopartnershare2      4589
cl_age                            0
gender_id                         1
tran_prod                       315
tran_remont                     315
tran_inet                       315
cred_tran_total             

In [275]:
df = df.fillna(0)

In [276]:
# разделим датасет на обучающую, валидационную и тестовую выборки
df_train_full, df_test = train_test_split(df, test_size=0.2, random_state=25)
df_train, df_val = train_test_split(df_train_full, test_size=0.33, random_state=25)

y_train = df_train.churn.values
y_val = df_val.churn.values

del df_train['churn']
del df_val['churn']

In [277]:
categorical = list(df_train.dtypes[df.dtypes == 'object'].index)
numerical = list(df_train.dtypes[df.dtypes != 'object'].index)

In [278]:
# применим one-hot-encoding
dv = DictVectorizer(sparse=False)

train_dict = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dict)

val_dict = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dict)

In [279]:
# Модель логистическая регрессия
lr = LogisticRegression(solver='liblinear', random_state=1)
lr.fit(X_train, y_train)

y_pred_lr = lr.predict_proba(X_train)[:, 1]
auc = roc_auc_score(y_train, y_pred_lr)
print('train auc: %.3f' % auc)

y_pred_lr = lr.predict_proba(X_val)[:, 1]
auc = roc_auc_score(y_val, y_pred_lr)
print('val auc: %.3f' % auc)

train auc: 0.494
val auc: 0.486


In [280]:
# Модель Random Forest
rf = RandomForestClassifier(max_depth = 20, min_samples_leaf = 5, n_estimators=180, random_state=3)
rf.fit(X_train, y_train)

y_pred_rf = rf.predict_proba(X_train)[:, 1]
auc = roc_auc_score(y_train, y_pred_rf)
print('train auc: %.3f' % auc)

y_pred_rf = rf.predict_proba(X_val)[:, 1]
auc = roc_auc_score(y_val, y_pred_rf)
print('val auc: %.3f' % auc)

train auc: 0.976
val auc: 0.770


In [281]:
# Подбор параметров Random Forest, наилучшая оценка модели
params = {
    "max_depth": [1, 2, 3, 4, 5, 6, 10, 15, 20],
    "min_samples_leaf": [1, 5, 10, 15, 20, 50, 100, 200]
}
gs = GridSearchCV(rf, params, scoring='roc_auc')
gs.fit(X_train, y_train)

GridSearchCV(estimator=RandomForestClassifier(max_depth=20, min_samples_leaf=5,
                                              n_estimators=180,
                                              random_state=3),
             param_grid={'max_depth': [1, 2, 3, 4, 5, 6, 10, 15, 20],
                         'min_samples_leaf': [1, 5, 10, 15, 20, 50, 100, 200]},
             scoring='roc_auc')

In [282]:
print('Best parameters: ' + str(gs.best_params_))
print('ROC-AUC ' + str(gs.best_score_))

Best parameters: {'max_depth': 20, 'min_samples_leaf': 5}
ROC-AUC 0.7749956428431469


In [283]:
# Модель градиентного бустинга XGBoost 
dtrain = xgb.DMatrix(X_train, label=y_train)
dval = xgb.DMatrix(X_val, label=y_val)

xgb_params = {
    'eta': 0.1,
    'max_depth': 3,
    'min_child_weight': 1,

    'objective': 'binary:logistic',
    'eval_metric': 'auc',
    'nthread': 8,
    'seed': 1,
}

model = xgb.train(xgb_params, dtrain,
                  num_boost_round=180)

In [284]:
y_pred_xgb = model.predict(dtrain)
auc = roc_auc_score(y_train, y_pred_xgb)
print('train auc: %.3f' % auc)

y_pred_xgb = model.predict(dval)
auc = roc_auc_score(y_val, y_pred_xgb)
print('val auc: %.3f' % auc)

train auc: 0.859
val auc: 0.771


In [285]:
# анализ feature_importances согласно модели Random Forest
importances = list(zip(dv.feature_names_, rf.feature_importances_))

df_importance = pd.DataFrame(importances, columns=['feature', 'gain'])
df_importance = df_importance.sort_values(by='gain', ascending=False)
df_importance.head(10)

,feature,gain
17,count_tran2,0.096922
74,tran_amt2,0.082053
18,counttran1,0.049455
26,free_limit2,0.043306
4,avg_amt_vnebs_3m,0.041450
73,tran_amt1,0.040687
76,tran_prod,0.035431
3,avg_amt_prod_3m,0.034198
23,credit_tran_amt_last_month,0.033432
72,tran_amt0,0.030639


In [286]:
# df_rfm = pd.read_csv(f'/content/in_actions.csv', index_col=0)
df_rfm = df_rfm.fillna(0)

In [287]:
main_features = [
    'count_tran2', 
    'tran_amt2',
    'counttran1',
    'tran_amt1',
    'tran_prod',
    'CREDIT_TRAN_AMT_last_month',
    'login_0',
    'login_1',
    'login_2',
    'tran_inet'
]

In [288]:
# проанализируем ушедших клиентов, определим условные сигнальные контрольные точки через медиану
df_exited = df_rfm[df_rfm.churn == 0]
df_exited[main_features].describe()

,count_tran2,tran_amt2,counttran1,tran_amt1,tran_prod,CREDIT_TRAN_AMT_last_month,login_0,login_1,login_2,tran_inet
count,7000.00000,7000.000000,7000.000000,7.000000e+03,7000.000000,7000.000000,7000.000000,7000.000000,7000.000000,7000.000000
mean,20.31500,19949.773180,23.442000,2.675600e+04,7.559857,11336.246511,10.691143,12.908429,12.667429,3.264714
std,21.87057,25772.320535,24.468531,3.475921e+04,12.086575,17717.724025,12.240274,13.610760,15.640809,8.397610
min,1.00000,0.440000,1.000000,9.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,6.00000,5496.075000,6.000000,7.822020e+03,1.000000,0.000000,3.000000,4.000000,4.000000,0.000000
50%,13.00000,12326.180000,16.000000,1.668219e+04,3.000000,4437.550000,7.000000,9.000000,10.000000,0.000000
75%,27.00000,25085.202500,32.000000,3.475736e+04,10.000000,15693.630000,14.000000,17.000000,16.000000,3.000000
max,233.00000,596125.220000,291.000000,1.033570e+06,171.000000,251096.600000,305.000000,195.000000,502.000000,150.000000


In [289]:
# проведем RFM сегментацию на имеющихся данных
# добавим столбцы R, F, M, характеризующие давность, частоту и общую стоимость
# добавим общий индекс RFM

In [290]:
# расчет частоты и стоимости

df_rfm = df_rfm.assign(frequency = df_rfm['count_tran0'] + df_rfm['counttran1'] + df_rfm['count_tran2'])
df_rfm = df_rfm.assign(monetary = df_rfm['tran_amt0'] + df_rfm['tran_amt1'] + df_rfm['tran_amt2'])

In [291]:
# Оценим распределение показателей RFM. Для разделения на группы по частоте и стоимости будем использовать перцентили.
# Группа 1: 0 - 25%
# Группа 2: 25 - 75%
# Группа 3: 75 - 100%
df_rfm[list(['count_tran2', 'frequency', 'monetary'])].describe()

,count_tran2,frequency,monetary
count,12988.000000,12988.000000,1.298800e+04
mean,14.845473,46.790499,7.425269e+04
std,18.997921,46.311910,7.786537e+04
min,1.000000,3.000000,3.000000e+01
25%,3.000000,15.000000,3.352569e+04
50%,8.000000,32.000000,5.710929e+04
75%,19.000000,62.000000,9.038397e+04
max,233.000000,556.000000,2.466512e+06


In [292]:
# Определение групп по давности. Используем данные порогового значения по количеству транзакций за 3 месяц. 
def calculate_r(series):
  if series == 1: 
    return 1
  elif 1 < series <= 13:
    return 2
  elif series > 13:
    return 3

In [293]:
# Определение групп по частоте
def calculate_f(series):
  if 0 < series <= 15:
    return 1
  elif 15 < series <= 62:
    return 2
  elif series > 62:
    return 3

In [294]:
# Определение групп по стоимости
def calculate_m(series):
  if 0 < series <= 33500:
    return 1
  elif 33500 < series <= 90000:
    return 2
  elif series > 90000:
    return 3

In [295]:
# Добавим данные в датасет
df_rfm['R'] = df_rfm['count_tran2'].apply(calculate_r).astype(str)
df_rfm['F'] = df_rfm['frequency'].apply(calculate_f).astype(str)
df_rfm['M'] = df_rfm['monetary'].apply(calculate_m).astype(str)

df_rfm['RFM'] = df_rfm.R + df_rfm.F + df_rfm.M

In [296]:
# оценим каких категорий пользователей большей всего.
df_rfm.RFM.value_counts().sort_values(ascending=False)

222    2005
333    1265
332    1172
221    1162
212    1006
322     975
223     809
211     794
112     679
111     490
321     474
323     434
232     312
213     287
122     280
233     208
331     159
113     145
121     128
123      99
132      43
231      34
133      25
131       3
Name: RFM, dtype: int64

In [297]:
# оценим и отсортируем коэффициенты корреляции для каждой из групп RFM

for group in df_rfm.RFM.unique():
  df_cor_rfm = df_rfm[df_rfm['RFM'] == group]
  df_cor_values = df_cor_rfm.drop(['churn'], axis=1).corrwith(df_cor_rfm.churn).to_frame('values')
  df_cor_values = df_cor_values.assign(sorted = df_cor_values['values'].abs())
  df_cor_values = df_cor_values.sort_values(['sorted'], ascending=False)
  print(group)
  print(df_cor_values.head(5))
  print(' ')

321
                             values    sorted
tran_prod                 -0.116960  0.116960
cred_tran_nopartnershare2  0.116592  0.116592
counttran1                -0.115627  0.115627
tran_amt0                  0.100098  0.100098
frequency                 -0.097579  0.097579
 
222
                             values    sorted
count_tran2               -0.210198  0.210198
cred_tran_nopartnershare0  0.178675  0.178675
FREE_LIMIT2               -0.128725  0.128725
cred_tran_nopartnershare1  0.122315  0.122315
cred_tran_nopartnershare2  0.121861  0.121861
 
333
                             values    sorted
cred_tran_nopartnershare0  0.189292  0.189292
count_tran2               -0.175462  0.175462
login_1                   -0.156986  0.156986
cred_tran_nopartnershare1  0.153102  0.153102
cred_tran_nopartnershare2  0.149764  0.149764
 
212
               values    sorted
frequency   -0.113909  0.113909
pro1        -0.108174  0.108174
pro0        -0.107974  0.107974
count_tran2 -0.091800 